<a href="https://colab.research.google.com/github/David-Delatorre/CECS-456-Project/blob/main/CECS456_ResNetV2_Model3_CNN_Project_V1_David_Delatorre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing libraries
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import backend as k
!pip install opendatasets
import opendatasets as od

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("andrewmvd/medical-mnist")

# print("Path to dataset files:", path)

# need kaggle username and password
od.download(
	"https://www.kaggle.com/datasets/andrewmvd/medical-mnist")


!pip install -Uqq fastai
from pathlib import Path
from fastai.metrics import error_rate
from fastai.vision.all import *
!pip install timm
import timm
import torch
import pickle
path=Path('../content/medical-mnist');
path

# List files in the dataset to verify
import os
print(os.listdir(path))

Mounted at /content/drive
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: daviddelatorre
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/medical-mnist


100%|██████████| 84.8M/84.8M [00:04<00:00, 18.1MB/s]



['CXR', 'Hand', 'BreastMRI', 'ChestCT', 'AbdomenCT', 'HeadCT']


In [3]:
import os
import shutil
from pathlib import Path
import random

# Define the paths to the subdirectories
root_path = path  # Change this to your actual path
categories = ['CXR', 'Hand', 'BreastMRI', 'ChestCT', 'AbdomenCT', 'HeadCT']

# Define paths for training, validation, and test sets
train_path = root_path / 'train'
valid_path = root_path / 'val'
test_path = root_path / 'test'

# Create directories if they don't exist
for split in [train_path, valid_path, test_path]:
    split.mkdir(parents=True, exist_ok=True)

# Create subdirectories within each split for each category
for split in [train_path, valid_path, test_path]:
    for category in categories:
        (split / category).mkdir(parents=True, exist_ok=True)

# Function to split images into train, val, and test sets
def split_data(source_path, train_path, valid_path, test_path, split_ratio=(0.6, 0.2, 0.2)):
    for category in categories:
        # Get all image paths in the category
        category_path = source_path / category
        images = list(category_path.glob('*'))  # All images in the category
        random.shuffle(images)  # Shuffle the images

        # Calculate split sizes
        num_images = len(images)
        train_size = int(num_images * split_ratio[0])
        valid_size = int(num_images * split_ratio[1])

        # Split the images
        train_images = images[:train_size]
        valid_images = images[train_size:train_size+valid_size]
        test_images = images[train_size+valid_size:]

        # Move images to the corresponding directories
        for img in train_images:
            shutil.move(img, train_path / category / img.name)
        for img in valid_images:
            shutil.move(img, valid_path / category / img.name)
        for img in test_images:
            shutil.move(img, test_path / category / img.name)

# Split the data
split_data(root_path, train_path, valid_path, test_path)


test_path = "/content/medical-mnist/test"


train_path="/content/medical-mnist/train"


val_path="/content/medical-mnist/val"


# using data augmentation to prevent overfitting and handle imbalance in dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
Image_gen = ImageDataGenerator(
        rescale = 1/255,
        shear_range=10,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=[0.5,2.0],
        width_shift_range = 0.2,
        rotation_range=20,
        fill_mode = 'nearest'
)
val_Datagen = ImageDataGenerator(
        rescale = 1/255
)

train = Image_gen.flow_from_directory(train_path, target_size=(224, 224),
                                       batch_size=32,
                                       class_mode='categorical'
                                       )

validation = Image_gen.flow_from_directory(val_path, target_size=(224, 224),
                                              batch_size=16,
                                              class_mode='categorical'
                                            )

test = val_Datagen.flow_from_directory(test_path, target_size=(224, 224),
                                       batch_size=16,
                                       class_mode='categorical'
                                      )

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=10)

reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                          patience=8)




import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50V2



# Load the pretrained ResNet model
resnet_base = ResNet50V2(
    weights='imagenet',  # Use weights pretrained on ImageNet
    include_top=False,   # Exclude the top layer
    input_shape=(224, 224, 3)  # Input shape (match your dataset)
)

# Freeze the base model
resnet_base.trainable = False

# Add custom classification head
model = models.Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),  # Pool features from the base model
    layers.Dense(256, activation='relu'),  # Fully connected layer
    layers.Dropout(0.5),  # Regularization
    layers.Dense(6, activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
)

history = model.fit(
    train,
    validation_data=validation,
    epochs=40,
    steps_per_epoch=100,       # Define steps per epoch explicitly
    validation_steps=32,       # Define validation steps explicitly
    callbacks=[early_stopping, reduce_learning_rate],
    shuffle=True
)

Found 35372 images belonging to 6 classes.
Found 11790 images belonging to 6 classes.
Found 11792 images belonging to 6 classes.
94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 83s 647ms/step - accuracy: 0.8376 - loss: 0.4638 - precision: 0.8776 - recall: 0.7938 - val_accuracy: 0.9922 - val_loss: 0.0226 - val_precision: 0.9922 - val_recall: 0.9902 - learning_rate: 0.0010
Epoch 2/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 58s 583ms/step - accuracy: 0.9836 - loss: 0.0510 - precision: 0.9836 - recall: 0.9836 - val_accuracy: 0.9902 - val_loss: 0.0280 - val_precision: 0.9902 - val_recall: 0.9902 - learning_rate: 0.0010
Epoch 3/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 63s 635ms/step - accuracy: 0.9872 - loss: 0.0384 - precision: 0.9871 - recall: 0.9865 - val_accuracy: 0.9980 - val_loss: 0.0106 - val_precision: 0.9980 - val_recall: 0.9980 - learning_rate: 0.0010
Epoch 4/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 516ms/step - accuracy: 0.9880 - loss: 0.0289 - precision: 0.9888 - recall: 0.9874 - val_accuracy: 0.9941 - val_loss: 0.0249 - val_precision: 0.9941 - val_recall: 0.9922 - learning_rate: 0.0010
Epoch 5/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 523ms/step - accura

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - accuracy: 0.9844 - loss: 0.0242 - precision: 0.9844 - recall: 0.9844 - val_accuracy: 0.9902 - val_loss: 0.0282 - val_precision: 0.9902 - val_recall: 0.9902 - learning_rate: 0.0010
Epoch 13/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 112s 499ms/step - accuracy: 0.9920 - loss: 0.0269 - precision: 0.9920 - recall: 0.9919 - val_accuracy: 0.9980 - val_loss: 0.0057 - val_precision: 0.9980 - val_recall: 0.9980 - learning_rate: 0.0010
Epoch 14/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 53s 538ms/step - accuracy: 0.9955 - loss: 0.0125 - precision: 0.9963 - recall: 0.9954 - val_accuracy: 0.9980 - val_loss: 0.0036 - val_precision: 0.9980 - val_recall: 0.9980 - learning_rate: 0.0010
Epoch 15/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 526ms/step - accuracy: 0.9881 - loss: 0.0331 - precision: 0.9881 - recall: 0.9881 - val_accuracy: 0.9922 - val_loss: 0.0402 - val_precision: 0.9922 - val_recall: 0.9922 - learning_rate: 0.0010
Epoch 16/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 527ms/step - acc

In [5]:
score = model.evaluate(train)

print("Train Loss: ", score[0])
print("Train Accuracy: ", score[1])
print("Train Precision: ", score[2])
print("Train Recall: ", score[3])


score = model.evaluate(test)
print("\nTest loss: ", score[0])
print("Test Accuracy: ", score[1])
print("Test Precision: ", score[2])
print("Test Recall: ", score[3])


1106/1106 ━━━━━━━━━━━━━━━━━━━━ 548s 494ms/step - accuracy: 0.9986 - loss: 0.0038 - precision: 0.9986 - recall: 0.9986
Train Loss:  0.00353241222910583
Train Accuracy:  0.9987843632698059
Train Precision:  0.9988124966621399
Train Recall:  0.9987277984619141
737/737 ━━━━━━━━━━━━━━━━━━━━ 31s 43ms/step - accuracy: 0.9993 - loss: 0.0018 - precision: 0.9993 - recall: 0.9993

Test loss:  0.002168550854548812
Test Accuracy:  0.9990671873092651
Test Precision:  0.9990671873092651
Test Recall:  0.9990671873092651
